# Further explanation of functions 
<br><br><br>

### zoom_and_pick_bregma ():
<br>

##### 1. The alternation between different brema points is done by changing the bregma_index
##### 2. Information about the bregma displayed on screen is on the cv2.putText in this function
##### 3. The width and height changes according to the scale_percent change and are used to create the resized_image. The resized_image is only useful for display. The cropping in UseROIStoSegment.ipynb is done on the original image with corrected dimensions of the polygon (see scaled_roi_points variable in user_drawing().
<br><br>
### draw_polygon ():
<br><br>

##### 1. To completely understand the arguments of this function, the documentation of cv2.SetMouseCallback must be studied.
##### 2. The drawing variable dictates if the user is in the phase of drawing, that is, still be pressing the left mouse click. As long as the user is pressing the left click, coordinates x and y are constantly collected (these are all coordinates of the polygon)
##### 3. The 'region' variable is delibarately not a list but a cycle from itertools in order to allow infinite alternation through elements. Therefore, the user will never reach to a final index.
<br><br>

### decide_on_drawing():
<br><br>
##### 1. It is important to note here that although this function is primarily created to make a decision about the drawing (save or retry) , it is running independently of whether a drawing has been created or not (Note that the exit option, the polylines and imshow are outside the 'if polygon is not None'.). This function calls cv2.SetMouseCallback when click events are detected, which by its turn calls draw_polygon. Practically, it provides the frame for the display of images and action on them. In sum, the function works like this : (a) check whether there is a polygon created (b) if so, save the ROI coordinates after you rescale them to the proportionate original size of image (c) while you are checking for (a), if there is not a polygon created and escape is pressed, exit the program.
##### 2. Every time the while loop breaks, there is the chance that the decide_on_drawing() will start again before the main loop of the main code jumps to the new image (for img_name in images_allnames :) . Whether the function will be called again inside this iteration of the main loop is depended on the retry variable. If the retry variable equals True, it means that the user has selected to re-draw the ROI. So the function is called again.
##### 3. The segmentation is done on a clone image because the function might be called several times if the user wants to re-draw. This way, the previous contours, bregma labels and region labels are also refreshed. 
##### 4. Note that if a polygon is created, it will not be None anymore but a Polygon instead. Which means that if the user decided to re-draw, that wouldn't be possible based on the first if condition under the while True loop (if polygon is not None). However, polygon is reset to None when the user wants to re-draw.
<br><br>

### define_bregmata() & define_regions():
##### These function will only be called if a 'Bregmata_and_regions.pkl' file does not exist on the workspace. By workspace we define the directory in which this file you read is. The functions prompt the user to submit the regions of interest and bregma points of interest that wants to use.

    

In [2]:
import cv2
import pickle
import os
import sys
import re
import numpy as np
from shapely.geometry import Polygon
from itertools import cycle, islice 
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askdirectory
from tkinter import messagebox
from os import listdir
from os.path import isfile, join, exists

def zoom_and_pick_bregma () :
    """ Only here there is the possibility to resize the image accordingly (by pressing a or d). 
        The user can alternate through the already registered bregma points by pressing q or e and select
        the desired one by pressing ampersand.
        """
    global image, resized_image, width, height, scale_percent, coordinates, bregmata, bregma_index, bregma_coordinates   # width and height are inserted because if no zoom in/out related button is pressed, there is no information about it
    final_bregma_decision = False
    while final_bregma_decision == False :                                                             # while ampersand is not preesed yet
        key = cv2.waitKey(0)
        if key == ord('d') :
            scale_percent = scale_percent + 0.1
            width, height = int(image.shape[1] * scale_percent / 100), int(image.shape[0] * scale_percent / 100)                                                                                 # Resize the image for better displ
            resized_image = cv2.resize(image, (width, height))
        if key == ord('a') :
            scale_percent = scale_percent - 0.1
            width, height = int(image.shape[1] * scale_percent / 100), int(image.shape[0] * scale_percent / 100)                                                                                 # Resize the image for better displ
            resized_image = cv2.resize(image, (width, height))
        if key == ord('q'):
            if bregma_index != 0 :
                bregma_index -= 1
            else :
                pass
        if key == ord('e'):
            try :
                bregmata[bregma_index+1]
            except IndexError :
                pass
            else :
                bregma_index += 1
        if key == 32 :
            final_bregma_decision = True                                                               # if & is pressed, decision has been made
            coordinates = (int(width*0.8),int(height*0.1))                                             # the region name will be displayed close to the end of width end start of height (that is top right)
            bregma_coordinates = (int(width*0.8),int(height*0.2))                                      # coordinates for the bregma
            cv2.destroyAllWindows()
            break 
            
        if key == 27:                                                                                  # exit program with escape
            cv2.destroyAllWindows()                                                                    # mind that this only works in the initial stage of bregma assessment. Won't work afterwards
            sys.exit(0)

        another_clone = resized_image.copy()                                                           # display the clone because resized_image can be overwhelmed with pixels (pixels will stay every time the bregma changes)
        cv2.putText(another_clone, bregmata[bregma_index], bregma_coordinates, font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow(f'Select zoom and bregma for {img_name}', another_clone)

def draw_polygon(event, x, y, flags, param):
    global drawing, polygon, roi_points, regions, region_label 
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        roi_points = [(x, y)]

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            roi_points.append((x, y))

    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            roi_points.append((x, y))
        drawing = False
        polygon = Polygon(roi_points)
        
    if event == cv2.EVENT_MOUSEWHEEL:
        region_label = next(regions)
        

def decide_on_drawing() :                                                                                   # This function will loop as long as retry = True
    """ The user decides whether (s)he saves the drawing or tries to draw again.
        If there is not a polygon drawn, there is the option to exit the program with escape button
        The rescaled coordinates of ROI (which are registered in the draw_polygon() are submitted to
        a dictionary (ROI_dic) which then becomes a pickle file. The dictionary is rewritable and 
        every time it is renewed the previous versions are truncated). The coordinates are rescaled because
        the display image is of different size than the original one."""
    global drawing, polygon, roi_points, retry 
    resized_window_name = 'Select ROI (Lasso)'
    cv2.namedWindow(resized_window_name)
    cv2.setMouseCallback('Select ROI (Lasso)', draw_polygon)                                           # looks for mouse events and registers them based on their type. Also keeps the coordinates of the mouse. It also provides a frame, although the image is not shown upon it yet at this stage.

    while True:
        clone = resized_image.copy()
        if polygon is not None:                                                                        # Calculate the scaling factors for resizing
            scale_x = image.shape[1] / resized_image.shape[1]
            scale_y = image.shape[0] / resized_image.shape[0]                                                   
            scaled_roi_points = []                                                                     # Scale the coordinates back to the original image size (to extract a proportionate space)
            for point in roi_points:
                scaled_x = int(point[0] * scale_x)
                scaled_y = int(point[1] * scale_y)
                scaled_roi_points.append((scaled_x, scaled_y))
            second_key = cv2.waitKey(0)
            
            if second_key == ord('s'):                                                                 # Exit the program if 'q' is pressed. If it's waitKey(1) it might not be detected. If not working try even larger values or press more quickly several times
                ROI_dic[animalID][region_label] = scaled_roi_points                                    # add the ROI and its coordinates to the dictionary. Example : 431934['bla']=... Note that 431934 has a dict as a value as defined in the global variables under the for loop of the main
                ROI_dic[animalID]['Bregma'] = bregmata[bregma_index]                                   # save the bregma point&&&&&
                with open('ROI_Coordinates_DONT_delete.pkl', 'wb') as writable:                        # the previous ROI_dic will be truncated and the new will take its place
                    pickle.dump(ROI_dic, writable)
                polygon = None                                                                         # reset to None to allow repetition. First the loop will break, but then the function will be recalled in the main code. And because polygon is None, the quit option will be available
                break                                                                                  # could also do continue but since the retry is still True it doesnt matter. The function will be recalled, hence the loop will be repeated
            if second_key == ord('r'):                                                                 # if it's waitKey(1) it might not be detected. If not working try even larger values or press more quickly several times      
                cv2.destroyAllWindows()
                drawing,polygon,roi_points,retry = False,None,[],True                                  # set the global variables to their default
                break
        if cv2.waitKey(2) == 38:                                                                       # Exit the program if escape is pressed. If it's waitKey(1) it might not be detected. If not working try even larger values or press more quickly several times
            cv2.destroyAllWindows()
            retry=False
            break
            
        cv2.polylines(clone, [np.array(roi_points)], isClosed=False, color=(255, 0, 255), thickness=3) # Note that this will work even during the creation of the contours, when polygon is still None (because there's no buttonup yet). roi_points is global, so as we draw the points, they are depicted.
        cv2.putText(clone, region_label, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.putText(clone, bregmata[bregma_index], bregma_coordinates, font, fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow(resized_window_name, clone)                                                         # By using the polylines, the contours are drawn anyway when we click and draw the region. However, we will only see the frame and not the image upon it unless we run this command. That's the way to see what we're doing
        cv2.setWindowTitle(resized_window_name, img_name) 



def showmessages (msg_type, title, message):
    defroot = Tk()
    defroot.geometry('10x10')
    if msg_type == 'warning':
        messagebox.showwarning(title=title, message=message)
    if msg_type == 'error':
        messagebox.showerror(title=title, message=message)
    else :
        print(f'Wrong msg_type. Must be warning or error but it was {msg_type}')
        print(type(msg_type))
    defroot.destroy()

        
def define_bregmata():
    def submit_bregmata() :
        global bregmata, list_with_breg_and_areas_for_pkl
        bregmata = brentry.get().split()
        bregmata.insert(0, 'Undefined bregma')
        list_with_breg_and_areas_for_pkl = [bregmata]                                                  # Create a list where the bregmata will be dumped inside. # Later on, in define regions(), the regions will also be appended to the list before it's finally dumped as a pickle.                                                  
        broot.destroy()
    broot = Tk()
    broot.title('Define bregmata')
    brentry = Entry(broot, width=40)
    brentry.grid(row=0, column=0, padx=30, pady=90) 
    brentry.insert(0, 'Type all bregma points separated by space')
    broot.resizable(True,True)
    broot.geometry('300x250')
    bButton = Button(broot,text = 'Submit', width = 5, command=submit_bregmata).place(x=120, y = 130)
    broot.mainloop()

def define_regions():
    def submit_regions() :
        global regions, region_label, listed_regions, list_with_breg_and_areas_for_pkl
        region_label = rrentry.get().split()[0]                                                        # Default label (Will only be used once, in the start of the program)
        regions,listed_regions = cycle(rrentry.get().split()),rrentry.get().split()                    #listed_regions will be used for creating subdirectories for saving each region if necessary
        list_with_breg_and_areas_for_pkl.append(regions)                                               # add the areas to the list where the bregmatas had been added
        if len(list_with_breg_and_areas_for_pkl) == 2:                                                 # one element should be the bregmata(from define bregmata)(type list) and the other one the regions (type cycle)                                 
            with open ('Bregmata_and_regions.pkl', 'wb') as writehere :
                pickle.dump(list_with_breg_and_areas_for_pkl, writehere)
            rroot.destroy()
        else :
            showmessages(msg_type ='error',title='Error in submission', message=' You probably omitted to define the regions or the bregma points. Please try again from the start.This program will terminate.')
            sys.exit(0)
    rroot = Tk()
    rroot.title('Define regions')
    rrentry = Entry(rroot, width=43)
    rrentry.grid(row=0, column=0, padx=23, pady=90) 
    rrentry.insert(0, 'Type all region abbreviations separated by space')
    rroot.resizable(True,True)
    rroot.geometry('300x250')
    rButton = Button(rroot,text = 'Submit', width = 5, command=submit_regions).place(x=120, y = 130)
    rroot.mainloop()    
    

def select_directory_frame ():
    def select_specific_directory():
        global latest_directory, directory_selected_at_least_once        
        latest_directory = filedialog.askdirectory(title='Folder with pkl images')                                                   # The folder with the default channel for each image must be selected here                                                                               
        directory_selected_at_least_once = True  
        root.destroy()
    root = Tk()
    root.title ('Select Images Folder')
    root.resizable(False,False)
    root.geometry('300x200')
    DirButton = Button(root,text='Locate your pkl images folder', command=select_specific_directory).place(x=75,y=80)
    root.mainloop()
    
def gather_directory_images():
    global latest_directory
    images_in_dir = [i for i in listdir(latest_directory) if isfile(join(latest_directory, i))]
    return images_in_dir

def retrieve_roi_coordinates():
    global ROI_dic
    if exists("ROI_Coordinates_DONT_delete.pkl") == True :
        with open('ROI_Coordinates_DONT_delete.pkl', 'rb') as readable:
            try :
                ROI_dic = pickle.load(readable)  
            except EOFError :
                showmessages(msg_type='error', title=' ', message='Your ROI coordinates file exists but might be empty. Try removing it from your workspace and running the program again. It is recommended to not delete it since it might contain information you need. This program will terminate.')
                sys.exit(0)
    else :
        showmessages(msg_type='warning', title=' ', message='ROI_Coordinates file not found in your workspace. The regions of interest will be registered from scratch in a new ROI coordinates file')
        ROI_dic = {}
        with open('ROI_Coordinates_DONT_delete.pkl', 'wb') as fp:
            pass
    
def find_or_create_subdir ():
    '''Creates subdirectories for saving images of each region. 
       Will only create a new subdirectory if it doesnt already exist
       listed_regions and region_label are set here if define_regions() is not called'''
    global listed_regions,region_label
    try :
        listed_regions[0]                                                                                # if the define region () has not been called because there was already a bregmata_and_regions file, the listed_regions won't exist. So we have to take it from the pickle
    except :
        with open ('Bregmata_and_regions.pkl', 'rb') as readreg:                                         # Note that the file that we open is actually the cycle (with itertools). So we have to convert back to list first
            listed_regions = pickle.load(readreg)[1]
            # convert back to list from cycle maximum number of 30 elements set. Change if necessary
            listed_regions= list(islice(listed_regions, 30))
            listed_regions = list(set(listed_regions))
            region_label = listed_regions[0]
            print('listed regions : ', listed_regions)

def start_from_checkpoint ():
    """Checks whether the selected folder directory has names of images that are already included
       in the ROI coordinates file. Takes the user back from where he stopped the segmentation 
       provided that he opens an unifinished directory which was used last."""
    global images_allnames
    for img_in_folder in images_allnames:
        if img_in_folder in ROI_dic.keys():
            showmessages(msg_type='warning', title='Warning', message='You have already segmented some images in your selected directory (see ROI_Coordinates_DONT_delete.pkl). You will be prompted to continue from the last processed image of your directory. If you wish to re-segment previously processed images, make sure that you create a new workspace directory with a copy of the current ipynb file.')
            last_registered_image = list(ROI_dic)[-1]
            if last_registered_image not in images_allnames :                                                # if the last coordinate submission to ROI(pickle) dictionary is not in this directory
                showmessages(msg_type='error', title='Error', message=f'It seems that the last image that was segmented is not included in your selected folder. Because the selected folder includes some already proccessed images, the execution of the program will stop. The last image processed was {last_registered_image}. Run the program again and try to use a folder containing this image to continue segmenting from where you stopped. If the problem persists, create a new workspace with a copy of this ipynb file.') # althoug it might seem inconvenient, this part ensures that the program will always keep runing from where it stopped
                sys.exit(0)
            else :
                last_checkpoint_position = images_allnames.index(last_registered_image)                      # finds where the last segmented and registered image was in the folder with the images to be segmented 
                try :
                    indexerror = images_allnames[last_checkpoint_position+1]                                 # only this will reveal if there is an index error. Slicing will not yield error by itself even if the list is empty and the indexes random.                   
                    images_allnames = images_allnames[last_checkpoint_position+1:]                           # and transforms the list with images to-be-segmented that are iterated in the main code's loop so that only images following the last processed image are included.  
                    return
                except IndexError :                                                                          # if there's nothing following, yield error. Note that if the user has re-sorted the files in the folder, it won't work properly. Because we assume that all images before the last registered image in the pickle file(dic) have been processed and all following images havent been processed. 
                    showmessages(msg_type= 'error', title='Error', message=f'The last segmented and registered image with the name {last_registered_image} is also the last image of this directory. You may want to start using another folder with new images. This program will terminate.')
                    sys.exit(0)
                    
def search_for_existing_bregmata_and_regions ():
    global bregmata,regions
    if exists('Bregmata_and_regions.pkl') == True:
        with open('Bregmata_and_regions.pkl', 'rb') as readpickle:
            pickle_with_bregma_and_regions = pickle.load(readpickle)
            if len(pickle_with_bregma_and_regions) == 2 :
                bregmata, regions = pickle_with_bregma_and_regions
                return True
            else :
                showmessages(msg_type= 'error', title='Wrong bregmata_and_regions.pkl', message='Your bregmata_and_regions.pkl file is not consisted of two elements (list with bregmata and list(cycle type) with regions). Delete the afforementioned file and define bregma points and regions again. If the problem persists try creating a new workspace with this ipynb file. This program will terminate.')
                sys.exit(0)
    else :
        return False


# variables related to the region and bregma label
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.75
color = (153,255,255)
thickness = 2 
scale_percent = 6                                                                                             # Default resize scale of the image for better display. The optimal variable may vary a lot depending on the dimensions of the image
# animalID_pattern = r'\d+_\d+x-\d+'                                                                            # Will isolate '427521_20x-01' from '427521_20x-01.czi-default.pkl'                           

retrieve_roi_coordinates()
if search_for_existing_bregmata_and_regions () == False:                                                      # If  there is no file that contains information about the regions and the bregma points in the workspace
    define_bregmata()
    define_regions()
select_directory_frame()
images_allnames = gather_directory_images()                                                                    # The folder with the default channel of each image must be the latest_directory
find_or_create_subdir()
start_from_checkpoint ()

                
for img_name in images_allnames :
    # Global variables
    if 'restchannels' in img_name :                                                                     # we want to iterate only the files that contain the word default because these are the images with the default channel
        continue
    drawing = False                                                                                     # the state where the left click has been pressed already and remains pressed
    polygon = None                                                                                      # based on shapely, a polygon containing all coordinates while drawing
    roi_points = []                                                                                     # every set of coordinates while drawing the polygon
    retry = True                                                                                        # when the user selects to save the image by pressing s, retry will become False and the loop of decide_on_drawing() will stop
    bregma_index = 0 
    whole_path = f'{latest_directory}/{img_name}'
    with open(whole_path, 'rb') as def_chan_img:
        image = pickle.load(def_chan_img)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)                                                      # re-converts the splitted image to BGR to allow colored contours and labels 
    image = cv2.convertScaleAbs(image, alpha=0.015, beta=15)
    width, height = int(image.shape[1] * scale_percent / 100), int(image.shape[0] * scale_percent / 100)                                                                                 
    resized_image = cv2.resize(image, (width, height))        
    bregma_coordinates = (int(width*0.5),int(height*0.15))
    animalID = img_name                                                
    ROI_dic[animalID]={"Subdirectory" : latest_directory}

    zoom_and_pick_bregma()
    while retry == True :                                                                                # will be set to false when ampersand is pressed by the user to indicate end of drawing on an image
        decide_on_drawing()

Wrong msg_type. Must be warning or error but it was warning
<class 'str'>


SystemExit: 0

c:\Users\angdid\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Notes to run correctly

#### First run the separatedefaultandrest to transform the images and separate the default from the rest

#### Then run the script here

#### Then run the UseROIsToSegment after you modify it to iterate



In [ ]:
4